stack

In [ ]:
from aws_cdk import Stack
from constructs import Construct

from infrastructure.constructs.data_scraper import DataScraper
from infrastructure.constructs.data_fetch_and_save import DataFetchAndSave


class DataScraperStack(Stack):
    def __init__(
        self,
        scope: Construct,
        construct_id: str,
        stage_name: str,
        service_config: dict,
        **kwargs,
    ) -> None:
        super().__init__(scope, construct_id, **kwargs)

        DataScraper(
            self,
            f"{stage_name}-DataScraper",
            stage_name=stage_name,
            service_config=service_config,
        )

        DataFetchAndSave(
            self,
            f"{stage_name}-DataFetchAndSave",
            stage_name=stage_name,
            service_config=service_config,
        )

lambda func

In [ ]:
import boto3
import os
from typing import Any, Dict
from aws_lambda_powertools.utilities.typing import LambdaContext
import json

dynamodb = boto3.resource('dynamodb')

def lambda_handler(event: Dict[str, Any], context: LambdaContext) -> Dict[str, Any]:
    table = dynamodb.Table('Cyrille-dscrap-bike-data-table')
    
    # Scanning the table
    response = table.scan(
        Limit=1,  # limit to one item
        AttributesToGet=['stationId'] # get only the stationId
    )
    
    if 'Items' in response and response['Items']: # if there is an item in the response
        station_id = response['Items'][0]['stationId'] # extract the value from first item in the result
    else:
        station_id = 'Not found'

    return {
        'statusCode': 200,
        'body': json.dumps(f'Station ID: {station_id}')
    }

# FUNKTIONEN SOM AWS KALLAR PÅ HÄR SKA VI SKRIVA KODEN FÖR ATT HÄMTA DATA OCH SPARA I S3

construc


In [ ]:
import aws_cdk
from aws_cdk import (
    aws_dynamodb,
    aws_lambda,
    aws_iam,
    Duration,
    aws_events,
    aws_events_targets,
)

#VIKTIGT: OBJEKT BLIR INSTATIERADE I BÖRJAN AV KODEN
# DET HÄR FÅR DET ATT SE UT SOM ATT EN VARIABEL BLIR KALLAD FÖRE DEN ÄR SKAPAD
# Funktioner i classer kallar för metoder

from constructs import Construct
import os


# Class definition and inheritance
class DataFetchAndSave(Construct): # inherits from Construct
    def __init__( # initialize the object and expect the following parameters
        self, scope: Construct, id_: str, stage_name: str, service_config: dict
    ) -> None:
        super().__init__(scope, id_) # Calls the parent class' constructor to initialize it

        # extracts the service name from the configuration
        service_short_name = service_config["service"]["service_short_name"]
        service_name = service_short_name

        # Formats the lambda name and role name
        lambda_name = f"{stage_name}-{service_short_name}-data-fetch-and-save"
        lambda_role_name = f"{lambda_name}-role"
        # "lambda_role" calls to get the role object for the lambda
        lambda_role = self._build_lambda_role(lambda_role_name)

        # environment variables for the lambda
        # configure the enviroment differently based on the stage or service
        #---------------------------------------------------------------------
        # env_vars = self._create_env_vars(
        #     stage_name=stage_name, service_name=service_name
        # )
        
        # builds lambda function - passes in the necessary configuration
        self.data_scraper_lambda = self._build_lambda(
            lambda_name=lambda_name,
            stage_name=stage_name,
            #env_vars=env_vars,
            lambda_role=lambda_role,
        )

    # creates an AWS Lambda function
    def _build_lambda(
        self,
        lambda_name: str, # name of the lambda
        stage_name: str, # Sets stage name, so i we know if its dev or prod etc
        #env_vars: dict, # environment variables
        lambda_role: aws_iam.Role, # Role - for permissions
        cwd: str = os.getcwd(), # current working directory
    ):
        lambda_function = aws_lambda.Function(
            self,
            lambda_name,# name that has logicalID in template.json
            function_name=lambda_name,
            runtime=aws_lambda.Runtime.PYTHON_3_10, # runtime setting - python 3.10
            #environment=env_vars,
            code=aws_lambda.Code.from_asset( # where the code for lambda function exists
                os.path.join(cwd, "bike_data_scraper/handlers")
            ),
            handler="data_fetch_and_save_lambda.lambda_handler",
            tracing=aws_lambda.Tracing.ACTIVE,
            retry_attempts=2,
            timeout=Duration.seconds(80), # time before lambda times out
            memory_size=128, # default memory size
            role=lambda_role,
        )
        return lambda_function
    
    
    def _build_lambda_role(self, role_name: str) -> aws_iam.Role:
            return aws_iam.Role(
                self,
                role_name,
                assumed_by=aws_iam.ServicePrincipal("lambda.amazonaws.com"),
                managed_policies=[
                    aws_iam.ManagedPolicy.from_aws_managed_policy_name(
                        managed_policy_name="service-role/AWSLambdaBasicExecutionRole"
                    )
                ],
            )
    
    # def _create_env_vars(self, stage_name: str, service_name: str):
    #     return {
    #         "STAGE_NAME": stage_name,
    #         "S3_BUCKET_NAME": "raw-data-weather-and-bikes",
    #         "LOG_LEVEL": "DEBUG",
    #         "SERVICE_NAME": service_name,
    #     }    

